In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
res=requests.get('https://www.ptt.cc/bbs/Food/M.1456820119.A.ACB.html',verify=False)
soup=BeautifulSoup(res.text)
practice=soup.select('#main-content')
print (practice)

[<div class="bbs-screen bbs-content" id="main-content"><div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">vul3nn (盛)</span></div><div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Food</span></div><div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[食記] 新竹東區-玉龍肉圓(原竹蓮肉圓)</span></div><div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Tue Mar  1 16:15:16 2016</span></div>
店名: 玉龍肉圓 (原竹蓮肉圓)
食用時間: 104/12
地址:  新竹市南大路464號（近新竹國小）
電話:  0910-960-776
營業時間:  08:00-20:00
價格:0-50
好讀網誌版
<a href="http://leosheng.tw/wunsylung/" rel="nofollow" target="_blank">http://leosheng.tw/wunsylung/</a>

南大路上的玉龍肉圓，靠近新竹國小，招牌上大大夫妻的照片，頗為醒目
看過去是老牌的小吃店，想說新竹老牌小吃店應該比隨意吃好吧，吃看看
玉龍肉圓算是竹蓮肉圓的接班店，第二代
竹蓮肉圓的原址由哥哥接手，弟弟則來到南大路上開這家玉龍
台灣各地不少小吃都是這樣，這難免是分家的關係
至於兩家有何差異真的是要吃看看才知道，"港款丟某剛師傅"道理就在這
新竹的肉圓常見比較多的是紅糟肉圓，應該都是用炸的
紅糟肉圓的味道跟中部

C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [4]:
res=requests.get('https://www.ptt.cc/bbs/Food/M.1457154609.A.2A3.html',verify=False)
soup=BeautifulSoup(res.text)
title=soup.select(".article-meta-value")[2].text   # id='#' and class='.'
article=soup.select(".article-meta-value")[0].text
times=soup.select('.article-meta-value')[3].text
content=soup.select('#main-content')[0].contents[4]

print (title)
print (article)
print(times)
print(content)

[食記] 台北/多麼Cafe+/早午餐、插座、不限時
uni929 (呱呱)
Sat Mar  5 13:10:07 2016


   餐廳名稱：多麼Cafe'+
   消費時間：2016年/3月
   地址：台北市大安區安和路217巷16號
   電話：02-2733-0009
   營業時間：週一 ～ 週四：11:30 - 23:00
             週五：11:30 - 1:00
             週六：10:30 - 1:00
             週日：10:30 - 23:00
   每人平均價位：450
   用餐時間：無限制
   插頭：每個位置都有，免費
   官網：


C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [ ]:
import requests
from bs4 import BeautifulSoup
def change_page():
    for i 

In [5]:
import requests, datetime, time, json
from bs4 import BeautifulSoup

def httpReq(board, indexNum):
    index_url = 'https://www.ptt.cc/bbs/' + board + '/index' + str(indexNum) + '.html'
    index_res = requests.get(index_url)
    return BeautifulSoup(index_res.text, 'html5lib')

def link_list(board, indexNum):
    link_list = []
    for link in httpReq(board, indexNum).select('.r-ent'):
        link_list.append('https://www.ptt.cc' + link.select('a')[0]['href'])
    return link_list

def grap_push_info(soup):
    pushInfo_list = []
    for i in soup.select('.push'):
        if len(i.select('.push-ipdatetime')[0].string.strip()) > 11:
            continue
        pushInfo_list.append(
                              # date (e.g. '02/18')
                             [i.select('.push-ipdatetime')[0].string.strip(), 
                              # userid (e.g. 'comdan66')
                              i.select('.push-userid')[0].string.strip(), 
                              # push content
                              i.select('.push-content')[0].string[1:].strip()]
        )
    return pushInfo_list

def grap_orig_post(soup):
    return (
        # date (e.g. 'Mon Aug 29 14:32:25 2005')
        soup.select('.article-meta-value')[3].text.strip() +'``!@'+
        # title
        soup.select('.article-meta-value')[2].text.strip() +'``!@'+
        # userid (e.g. 'skyboy')
        soup.select('.article-meta-value')[0].text.split('(')[0].strip() +'``!@'+                   
        # content
        soup.select('#main-content')[0].contents[4].strip()
    ).split('``!@')

def get_year(soup, push_m, push_d):
    m_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
    
    post_y = int(soup.select('.article-meta-value')[3].text.strip()[-4:])
    post_m = m_dict[soup.select('.article-meta-value')[3].text.strip()[4:7].lower()]
    
    if push_m < post_m:
        return str(post_y+1)
    else:
        return str(post_y)

def crawl_in_page(board, indexNum):
    for link in link_list(board, indexNum):
        try:
            con_res = requests.get(link)
            con_soup = BeautifulSoup(con_res.text, 'html5lib')
        except:
            counter = 1
            while counter <= 3:
                print('Retry after 2 sec...')
                time.sleep(2)
                con_res = requests.get(link)
                con_soup = BeautifulSoup(con_res.text, 'html5lib')
                counter += 1
        try:
            # crawl main content
            ptt_data.append({
                'date'   : grap_orig_post(con_soup)[0],
                'title'  : grap_orig_post(con_soup)[1],
                'userid' : grap_orig_post(con_soup)[2],
                'content': grap_orig_post(con_soup)[3],
                'url'    : con_res.url,
                'is_orig_post': True
            })

            # crawl push content
            for i in grap_push_info(con_soup):

                ptt_data.append({
                    'date'   : get_year(con_soup, int(i[0][0:2]), int(i[0][3:5])) + '/' + i[0],
                    'title'  : grap_orig_post(con_soup)[1],
                    'userid' : i[1],
                    'content': i[2],
                    'url'    : con_res.url,
                    'is_orig_post': False
                })

            with open('ptt_data.json', 'w') as outfile:
                json.dump(ptt_data, outfile)
            
        except:
            continue
            

ptt_data = []
board = ['Yunlin', 'Lifeismoney']
indexCounter = 1
boardCounter = 0
crawl_in_page(board[boardCounter], indexCounter)

#while True:
#    try:
#        crawl_in_page(board[boardCounter], indexCounter)
#    except:
#        crawl_in_page(board[boardCounter+1], indexCounter)
#        break

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [6]:
import requests
import datetime
import time
from bs4 import BeautifulSoup

def get_indexPage(board, indexNum):
    retry_counter = 1
    while retry_counter <= 3:
        index_url = 'https://www.ptt.cc/bbs/' + board + '/index' + str(indexNum) + '.html'
        try:
            return requests.get(index_url, verify=False)
        except KeyboardInterrupt:
            print('Interupted by operator.')
            break
        except:
            print('Retry after 2 sec ...')
            time.sleep(2)
            return requests.get(index_url, verify=False)
        retry_counter += 1
    return '1'

def crawl_data(record):
    try:
        return (record.select('.author')[0].text.strip() + '``' +
                record.select('a')[0].text.strip() + '``' +
                'https://www.ptt.cc' + record.select('a')[0]['href'])
    except IndexError:
        return (record.select('.title')[0].text.strip() + '``'
                + 'None')

def crawl_timestamp(record):
    ts_url = 'https://www.ptt.cc' + record.select('a')[0]['href']
    try:
        ts_res = requests.get(ts_url, verify=False)
    except IndexError:
        print('Retry after 2 sec ...')
        time.sleep(2)
        ts_res = requests.get(ts_url, verify=False)
    ts_soup = BeautifulSoup(ts_res.text, 'lxml')
    return ts_soup.select('.article-meta-value')[3].text[4:].replace('：',':').strip()



indexNum = 1
while indexNum <= 1:
#while True:
    try:
        board = 'Yunlin'
        soup = BeautifulSoup(get_indexPage(board, indexNum).text, 'lxml')
    except:
        continue

    for record in soup.select('.r-ent'):
        try:
            crawl_timestamp(record)
        except:
            continue
        print((crawl_timestamp(record) +'``'+
               crawl_data(record) + '``' +
               board + '``' +
               str(indexNum)
              ).split('``'))

    indexNum += 1

print('Nothing More...')

#datetime.strptime(crawl_timestamp(record), '%b %d %H:%M:%S %Y')

C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


['Aug 29 14:32:25 2005', 'skyboy', ':)歡迎光臨', 'https://www.ptt.cc/bbs/Yunlin/M.1125296921.A.E57.html', 'Yunlin', '1']
['Aug 29 22:17:34 2005', 'smalloswald', '我來遲了', 'https://www.ptt.cc/bbs/Yunlin/M.1125324964.A.690.html', 'Yunlin', '1']

C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)



['Aug 30 22:29:00 2005', 'skyboy', '[公告] 萬年版龜 v1.0', 'https://www.ptt.cc/bbs/Yunlin/M.1125411883.A.A4A.html', 'Yunlin', '1']

C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


...


['Aug 31 20:19:29 2005', 'yl871015', '[自介] YL871015', 'https://www.ptt.cc/bbs/Yunlin/M.1125490493.A.91F.html', 'Yunlin', '1']

C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)



['Aug 31 22:12:26 2005', 'ymam', '[自介] YMAM', 'https://www.ptt.cc/bbs/Yunlin/M.1125497333.A.D2D.html', 'Yunlin', '1']


C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user1\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


IndexError: list index out of range

TypeError: bad operand type for unary -: '_Helper'